# Power Mains Monitoring (a PoP type and crontab task)

This monitor script is used for the AC Power Sensor PoP type.

The following issues are used in tickets opened by this script:
- CODE_RED
- CODE_GREEN

### CODE_RED MAINS POWER FAILURE<br>
This issue can arise if a circuit breaker has tripped.,
### CODE_GREEN MAINS POWER OK<br>
This is the normal status for all monitored circuits.

In [1]:
DEPLOYED_ACTIVE  = 10001

#Ticket Types
CODE_GREEN  = 11000  # normally not ticketed
CODE_RED    = 11004

In [2]:
import os
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms')
print("============================================")
print("/  Power Mains is running.                  /")
print("============================================")
import requests
from datetime import datetime, timedelta
import pytz
from slackclient import SlackClient
import json
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser
import time

print(os.getcwd())
config = configparser.ConfigParser()
config.read("../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']
    
  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])
      
def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

/  Power Mains is running.                  /
/home/sensei/jupy-notebooks/Analytics/PorterFarms
Python version
3.7.2 (default, Dec 29 2018, 06:19:36) 
[GCC 7.3.0]
Version info.
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


In [3]:
def ticketIssue(conn, node_id, ticket_type, description):
  openTicket = kt.ticketExists(conn, node_id, ticket_type, [kt._OPEN_STATUS, kt._WORKING_STATUS])
  if openTicket is None:
    query = "SELECT    node.name, node.location_id, location.description, location.slackalertchannel_id, \
                       location.imageurl, customer.slacktoken, slackchannel.idslackchannel, slackchannel.channelid \
                       FROM kanji_node node JOIN kanji_location location ON location.idlocation=node.location_id \
                       JOIN kanji_slackchannel slackchannel ON location.slackalertchannel_id=slackchannel.idslackchannel \
                       JOIN kanji_customer customer ON customer.idcustomer=node.customer_id \
                       WHERE node.idnode={}".format(node_id)
    logger(_LOG_INFO, query)
    df = pd.read_sql(query, conn)    
    locationid = df['location_id'][0]
    locationdescription = df['description'][0]
    alerttextquery = "SELECT alerttext FROM kanji_tickettype WHERE idtickettype={}".format(ticket_type)
    df2 = pd.read_sql(alerttextquery, conn)
    ticketdescription = description + " " + df2['alerttext'][0]
    slackchannelid = df['idslackchannel'][0]
    slackchannelname = df['channelid'][0]
    nodename = df['name'][0]
    mentions = "@Charlie"
    locationimageurl = df['imageurl'][0]
    slacktoken = df['slacktoken'][0]
    
    logger(_LOG_DEBUG,"channel={} token={}".format(slackchannelid,slacktoken))
    
    ticketid = kt.openticket(conn, node_id, locationid, ticketdescription, 2, 3, ticket_type, slackchannelid)
    ts = kt.slackticket(nodename, locationdescription, ticketdescription, mentions, 2, 3, locationimageurl, \
                        slacktoken,slackchannelname, ticketid, 0)
    kt.updateTicket(conn, ticketid, ts)  
    logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
  else:
    logger(_LOG_INFO, "There is an existing ticket {} for this issue. {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))

In [4]:
messagetemplate = "[\
   {\"type\": \"section\", \
		\"text\": { \
			\"type\": \"mrkdwn\", \
			\"text\": \"*<fakeLink.toUserProfiles.com|Iris / Zelda 1-1>*\\nTuesday, January 21 4:00-4:30pm\\nBuilding 2 - Havarti Cheese (3)\\n2 guests\" \
		}, \
		\"accessory\": { \
			\"type\": \"image\", \
			\"image_url\": \"https://api.slack.com/img/blocks/bkb_template_images/notifications.png\", \
			\"alt_text\": \"calendar thumbnail\" \
		} \
   } ]"

In [5]:
def postMessageToSlack(text):    
    sc = SlackClient(_SLACK_TOKEN)
    slackchannel = "infrastructure"
    response = sc.api_call("chat.postMessage", channel=slackchannel, text=text, blocks=[])
    if not 'ok' in response or not response['ok']:
      print("Error posting message to Slack channel")
      print(response)
    else:
      print("Ok posting message to Slack channel")   

# Strategy
Query the AC Power Sensor PoP type on ALL deployed motes.

Query the poplog for the sensor value.  Restrict the query to a single value in the previous 5 minute period. If found, determine the status of the monitored circuit.

The mains sensor value range is 0 to 10.  A mains sensor value <5 indicates Power Ok.  A mains sensor value 10 is indicative of a power failure

In [10]:
_LOG_LEVEL = _LOG_DEBUG

_AC_POWER_SENSOR_POPTYPE = 10002

_MAINS_THRESHOLD_VALUE = 7

popquery = "SELECT pop.idpop, node.idnode, node.name, location.description as location \
            FROM kanji_pop pop \
            JOIN kanji_node node ON node.idnode=pop.node_id \
            JOIN kanji_location location on location.idlocation=node.location_id \
            WHERE type_id={} AND deploystate_id ={}".format(_AC_POWER_SENSOR_POPTYPE, DEPLOYED_ACTIVE)

logger(_LOG_DEBUG, "popquery={}".format(popquery))
pops = pd.read_sql(popquery, conn)
if pops.size>0:
  for popidx in pops.index:
    motename = pops['name'][popidx]
    moteid = pops['idnode'][popidx]
    motelocation = pops['location'][popidx]
    popid = pops['idpop'][popidx]
    print("mote = {} {} at {}".format(motename, moteid, motelocation))
    popvalquery = "SELECT timestamp, value FROM kanji_poplog \
                     WHERE pop_id={} AND \
                     timestamp > now() - INTERVAL '{} minutes' \
                     ORDER BY timestamp desc LIMIT 1".format(popid, 5)
    vals = pd.read_sql(popvalquery, conn)
    if vals.size>0: 
      for validx in vals.index:
         value = vals['value'][validx]
         timestamp = vals['timestamp'][validx]
         print("value={}".format(value))
         if (value > _MAINS_THRESHOLD_VALUE):
            print("Power Failure at {}".format(timestamp))
            #ticketIssue(conn, moteid, CODE_RED, "CODE_RED Power Failure on PoP {}.".format(popid))
            #postMessageToSlack("{} CODE_RED Power Failure on PoP {}.".format(motelocation, popid))
         else:
            print("PowerOk at {}".format(timestamp))
    else:
      print("Query {} returned no results".format(popvalquery))            

popquery=SELECT pop.idpop, node.idnode, node.name, location.description as location             FROM kanji_pop pop             JOIN kanji_node node ON node.idnode=pop.node_id             JOIN kanji_location location on location.idlocation=node.location_id             WHERE type_id=10002 AND deploystate_id =10001
mote = agMote-20010 20010 at Breeding
value=2.0
PowerOk at 2022-06-10 14:29:08.786301+00:00
